In [1]:
import pandas as pd
import orjson as json

In [2]:
for line in open('../data/train_matches.jsonl'):
    game = json.loads(line)
    for player in game['players']:
        break
    break

In [3]:
def player_kill_streaks(player):
    
    ks_cum = {'3-4': 0, '5-6': 0, '>6': 0}

    ks = {int(k):v for k, v in player['kill_streaks'].items()}
    sorted_ks = sorted(ks.keys(), reverse=True)

    for i, k in enumerate(sorted_ks):
        if ks[k] == 0:
            continue
        if k < 5:
            ks_cum['3-4'] += ks[k]
        elif k < 7:
            ks_cum['5-6'] += ks[k]
        else:
            ks_cum['>6'] += ks[k]
        v = ks[k]
        for ki in sorted_ks[i:]:
            ks[ki] -= v
    return ks_cum

def player_multi_kills(player):
    
    mk_cum = {'2': 0, '>2': 0}

    mk = {int(k):v for k, v in player['multi_kills'].items()}
    sorted_mk = sorted(mk.keys(), reverse=True)

    for i, k in enumerate(sorted_mk):
        if mk[k] == 0:
            continue
        if k == 2:
            mk_cum['2'] += mk[k]
        else:
            mk_cum['>2'] += mk[k]
        v = mk[k]
        for ki in sorted_mk[i:]:
            mk[ki] -= v
    return mk_cum

def team_kill_streaks_features(game):
    if 'init' not in dir(team_kill_streaks_features):
        multi_kills = ['2', '>2']
        kill_streaks = ['3-4', '5-6', '>6']
        team_kill_streaks_features.init = {'r_kill_streaks': 0, 'r_multi_kills': 0,
                                           'd_kill_streaks': 0, 'd_multi_kills': 0}
        team_kill_streaks_features.init.update({f'r{i}_multi_kills_{mk}': 0 for mk in multi_kills for i in range(1, 6)})
        team_kill_streaks_features.init.update({f'd{i}_multi_kills_{mk}': 0 for mk in multi_kills for i in range(1, 6)})
    
        team_kill_streaks_features.init.update({f'r{i}_kill_streaks_{mk}': 0 for mk in kill_streaks for i in range(1, 6)})
        team_kill_streaks_features.init.update({f'd{i}_kill_streaks_{mk}': 0 for mk in kill_streaks for i in range(1, 6)})

    
    c = team_kill_streaks_features.init.copy()
    
    for i, player in enumerate(game['players']):
        team = 'r' if i < 5 else 'd'
        idx = i % 5 + 1
        
        mk_cum = player_multi_kills(player)
        for k, v in mk_cum.items():
            c[f'{team}{idx}_multi_kills_{k}'] = v
        c[f'{team}_multi_kills'] += sum(mk_cum.values())

        ks_cum = player_kill_streaks(player)
        for k, v in ks_cum.items():
            c[f'{team}{idx}_kill_streaks_{k}'] = v
        c[f'{team}_kill_streaks'] += sum(ks_cum.values())
    return c

In [4]:
team_kill_streaks_features(game)

{'r_kill_streaks': 0,
 'r_multi_kills': 0,
 'd_kill_streaks': 0,
 'd_multi_kills': 0,
 'r1_multi_kills_2': 0,
 'r2_multi_kills_2': 0,
 'r3_multi_kills_2': 0,
 'r4_multi_kills_2': 0,
 'r5_multi_kills_2': 0,
 'r1_multi_kills_>2': 0,
 'r2_multi_kills_>2': 0,
 'r3_multi_kills_>2': 0,
 'r4_multi_kills_>2': 0,
 'r5_multi_kills_>2': 0,
 'd1_multi_kills_2': 0,
 'd2_multi_kills_2': 0,
 'd3_multi_kills_2': 0,
 'd4_multi_kills_2': 0,
 'd5_multi_kills_2': 0,
 'd1_multi_kills_>2': 0,
 'd2_multi_kills_>2': 0,
 'd3_multi_kills_>2': 0,
 'd4_multi_kills_>2': 0,
 'd5_multi_kills_>2': 0,
 'r1_kill_streaks_3-4': 0,
 'r2_kill_streaks_3-4': 0,
 'r3_kill_streaks_3-4': 0,
 'r4_kill_streaks_3-4': 0,
 'r5_kill_streaks_3-4': 0,
 'r1_kill_streaks_5-6': 0,
 'r2_kill_streaks_5-6': 0,
 'r3_kill_streaks_5-6': 0,
 'r4_kill_streaks_5-6': 0,
 'r5_kill_streaks_5-6': 0,
 'r1_kill_streaks_>6': 0,
 'r2_kill_streaks_>6': 0,
 'r3_kill_streaks_>6': 0,
 'r4_kill_streaks_>6': 0,
 'r5_kill_streaks_>6': 0,
 'd1_kill_streaks_3-4': 